In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers

In [2]:
class CustomAttention(layers.Layer):
    def __init__(self, **kwargs):
        super(CustomAttention, self).__init__(**kwargs)
    
    def build(self, input_shape):
        print('input_shape={}'.format(input_shape))
        self.kernel = self.add_weight(
            shape=(1, *input_shape[1:-1], 1),
            initializer=initializers.Zeros(),
            name="kernel",
            trainable=True,
        )
    
    def call(self, inputs):
        return tf.multiply(inputs, tf.math.sigmoid(self.kernel)*2)

In [5]:
inputs = keras.Input(shape=(4098,1))

crop = layers.Cropping1D((0,2))(inputs)
reshape = layers.Reshape((512,8))(crop)

x = CustomAttention()(reshape)

# Conv unit 1
conv = layers.Conv1D(16, kernel_size=3, strides = 1, padding = 'same',name = "conv1")(x)
bn = layers.BatchNormalization(name = 'bn1')(conv)
relu = layers.ReLU(name = 'relu1')(bn)
maxpool = layers.MaxPooling1D(pool_size = 2, strides = 2, name = 'max1')(relu)


# Conv unit 2
conv = layers.Conv1D(16, kernel_size=3, strides = 1, padding = 'same',name = "conv2")(maxpool)
bn = layers.BatchNormalization(name = 'bn2')(conv)
relu = layers.ReLU(name = 'relu2')(bn)
maxpool = layers.MaxPooling1D(pool_size = 2, strides = 2, name = 'max2')(relu)


# Conv unit 3
conv = layers.Conv1D(32, kernel_size=3, strides = 1, padding = 'same',name = "conv3")(maxpool)
bn = layers.BatchNormalization(name = 'bn3')(conv)
relu = layers.ReLU(name = 'relu3')(bn)
maxpool = layers.MaxPooling1D(pool_size = 2, strides = 2, name = 'max3')(relu)


# Conv unit 4
conv = layers.Conv1D(32, kernel_size=3, strides = 1, padding = 'same',name = "conv4")(maxpool)
bn = layers.BatchNormalization(name = 'bn4')(conv)
relu = layers.ReLU(name = 'relu4')(bn)
maxpool = layers.MaxPooling1D(pool_size = 2, strides = 2, name = 'max4')(relu)
drop = layers.Dropout(0.1,name = 'drop4')(maxpool)


# Conv unit 5
conv = layers.Conv1D(64, kernel_size=3, strides = 1, padding = 'same',name = "conv5")(maxpool)
bn = layers.BatchNormalization(name = 'bn5')(conv)
relu = layers.ReLU(name = 'relu5')(bn)
maxpool = layers.MaxPooling1D(pool_size = 2, strides = 2, name = 'max5')(relu)
drop = layers.Dropout(0.2,name = 'drop5')(maxpool)


# Conv unit 6
conv = layers.Conv1D(64, kernel_size=3, strides = 1, padding = 'same',name = "conv6")(drop)
bn = layers.BatchNormalization(name = 'bn6')(conv)
relu = layers.ReLU(name = 'relu6')(bn)
maxpool = layers.MaxPooling1D(pool_size = 2, strides = 2, name = 'max6')(relu)


# flatten, dense and softmax
flat = layers.Flatten(name = 'flat')(maxpool)
dense = layers.Dense(512, name = 'dense1')(flat)
drop = layers.Dropout(0.5,name = 'drop1')(dense)
prediction = layers.Dense(1, name = 'dense',activation = 'sigmoid')(drop)

model = keras.Model(inputs=inputs, outputs=prediction, name = 'model2')

input_shape=(None, 512, 8)


In [6]:
model.summary()
#model.save("model_dense_drop3.h5")

Model: "model2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4098, 1)]         0         
_________________________________________________________________
cropping1d_1 (Cropping1D)    (None, 4096, 1)           0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 512, 8)            0         
_________________________________________________________________
custom_attention_1 (CustomAt (None, 512, 8)            512       
_________________________________________________________________
conv1 (Conv1D)               (None, 512, 16)           400       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 512, 16)           64        
_________________________________________________________________
relu1 (ReLU)                 (None, 512, 16)           0    

In [6]:
#keras.utils.plot_model(model, "model1.png")